# Setup STL calibration table

## Find tag crop positions

In [ ]:
from time import sleep
import sys

sys.stderr = sys.__stderr__  # redirect stderr back to the shell

from pupil_apriltags import Detector
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.trajectories.stl_table import STLCalibrationTable, make_im_slice
from skrgbd.devices.rig import Rig
from skrgbd.devices.robot.robot_on_table import RobotOnSTLTable

In [ ]:
robot = RobotOnSTLTable()
trajectory = STLCalibrationTable(robot)

In [ ]:
rig = Rig(stl_right=True, phone_left=False, phone_right=False)
rig.init_cameras().join()
rig.start_cameras().join()

In [ ]:
detector = Detector(
    families='tag36h11',
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.25,
    debug=0
)

In [ ]:
# image = rig.stl_right.snap_frame()['image']
# image = rig.realsense.snap_frame()['image']
# image = rig.tis_left.snap_frame()['image']
# image = rig.tis_right.snap_frame()['image']
image = rig.kinect.snap_frame()['image']

if image.ndim > 2:
    image = image[..., 1]  # keep only green channel since tag detector only works with single-channel images
plt.imshow(image, cmap='gray')

In [ ]:
detection = detector.detect(image)[0]

x, y = detection.center
size = (detection.corners[0, 0] - detection.center[0]) * 2

size = size * 1.3
size = round(size)
x = round(x)
y = round(y)

print(f'({y}, {x}), {size}')
plt.imshow(image[make_im_slice([y, x], size)], cmap='gray')